In [17]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1,3"



%matplotlib inline
import matplotlib.pyplot as plt


import argparse
from torchvision import transforms
from model.lstm_pm import LSTM_PM
from data.handpose_data import UCIHandPoseDataset

import torch
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR

from torch.autograd import Variable
from torch.utils.data import DataLoader
import os

import scipy.misc
from IPython.core.debugger import set_trace
import json
import numpy as np

In [10]:
nb_temporal = 4
nb_epochs=20
batch_size=4

In [11]:
transform = transforms.Compose([transforms.ToTensor()])

data_dir = 'dataset/frames/combine_hand/combine_hand'
label_dir = 'dataset/label/combine_label/combine_label'
dataset = UCIHandPoseDataset(data_dir=data_dir, label_dir=label_dir, temporal=nb_temporal)
train_dataset = DataLoader(dataset, batch_size=batch_size, shuffle=True)
net = LSTM_PM(T=nb_temporal).cuda()

In [15]:
def loss_history_init():
    loss_history={}
    for i in range(batch_size):
        loss_history['train'+str(i+1)]=[]
        loss_history['test'+str(i+1)] = []
    loss_history['train'] = []
    loss_history['test'] = []
    return loss_history

In [ ]:
def save_image(save_path, nb_temporal, pred_heatmap, label_map=None, size=45, nb_heatmap=22):
    # predict_heatmaps shape: [nb_temporal, batch_size,22,45,45]
    for i in range(pred_heatmap[0].shape[0]):# each batch (person)
        if label_map==None:
            s = np.zeros((size, size*nb_temporal)) 
        else:
            s = np.zeros((size*2, size*nb_temporal)) 
            
        for j in range(len(pred_heatmap)):  # each temporal
            for k in range(nb_heatmap):
                if label_map==None:
                    s[:, j*45:(j+1)*45]+=predict_heatmaps[j].cpu().data.numpy()[i,k,:,:]
                else:
                    s[:45, j*45:(j+1)*45]+=label_map[i , j, k, :, :]
                    s[45:, j*45:(j+1)*45]+=predict_heatmaps[j].cpu().data.numpy()[i,k,:,:]
        scipy.misc.imsave(save_path+str(i+1)+'.jpg', s)

In [16]:
loss_history = loss_history_init()
save_json_path = "train_history.json"
save_weight_path = "./checkpoint/penn_lstm_pm_"
save_runtime_heatmap_path = "runtime_heatmaps/"

In [ ]:
optimizer = optim.SGD(params=net.parameters(), lr=8e-6, momentum=0.9, weight_decay=5e-4)
scheduler = StepLR(optimizer, step_size=40000, gamma=0.333)
criterion = nn.MSELoss()


for epoch in range(nb_epochs):
    print 'epoch......................' + str(epoch+1)
    net.train(True)
    for idx, (tmp1, tmp2, center_map) in enumerate(train_dataset):
        if tmp2.shape[0]!=batch_size:
            break           
        images = tmp1
        label_map = tmp2
        
        optimizer.zero_grad()
        predict_heatmaps = net(images, center_map)  # list
        train_loss = 0
        for i in range(len(predict_heatmaps)):
            predict = predict_heatmaps[i]# each batch(person)
            target = label_map[:,i, :, :, :]
            tmp_loss = criterion(predict, target) #loss of each temporal
            loss_history['train'+str(i+1)].append(float(tmp_loss))
            train_loss += tmp_loss * batch_size

        loss_history['train'].append(float(loss))
        train_loss.backward()
        ## *******************************************
        optimizer.step()
        scheduler.step()
        if idx%10==9:
            print "Epoch: "+str(epoch+1)+" itr: "+str(idx+1)+" loss: "+str(float(loss))
        
#         if idx==10:
#             break
    #save loss history and parameters after each epoch
    json.dump(loss_history, open(save_json_path, 'wb')) 
    
    if epoch%50==0: #save parameters
        torch.save(net.state_dict(),save_weight_path+str(epoch))
    
    #***********************save train heatmap after each 20 epoch**********************************
    if epoch%20 == 0:
        tmp_path = save_runtime_heatmap_path+'/'+'train'+'/e'+str(epoch)+'_bat'
        save_image(tmp_path, nb_temporal, predict_heatmaps, label_map)
    

    #*****************************test after each epoch *****************************************
    net.eval()
    test_loss = 0
    for stp in range(int(len(x.shape[0])/batch_size)+1):
        #load test data
    
        predict_heatmaps = net(testx_batch, center_map_test)
        for i in range(len(predict_heatmaps)):
            predict = predict_heatmaps[i]# each batch(person)
            target = label_map[:,i, :, :, :]
            # print target.shape
            tmp_loss = criterion(predict, target)
            loss_history['test'+str(i+1)].append(tmp_loss)
            test_loss += tmp_loss * testy_batch.shape[0]          
        loss_history['test'].append(test_loss)
        
        if epoch%20 == 0:  # save image
            tmp_path = save_runtime_heatmap_path+'/'+'test'+'/e'+str(epoch)+'_bat'
            save_image(tmp_path, nb_temporal, predict_heatmaps, label_map)
            

epoch......................1
Epoch: 1 itr: 1 loss: 0.00296654156409
Epoch: 1 itr: 2 loss: 0.00290113128722
Epoch: 1 itr: 3 loss: 0.00232817674987
Epoch: 1 itr: 4 loss: 0.0023020291701
Epoch: 1 itr: 5 loss: 0.00266363937408
Epoch: 1 itr: 6 loss: 0.00271297967993
Epoch: 1 itr: 7 loss: 0.00241631129757
Epoch: 1 itr: 8 loss: 0.00215244945139
Epoch: 1 itr: 9 loss: 0.00286916038021
Epoch: 1 itr: 10 loss: 0.0024396693334
Epoch: 1 itr: 11 loss: 0.00239616958424
epoch......................2


/home/danningx/.conda/envs/handpose-py2/lib/python2.7/site-packages/ipykernel_launcher.py:65: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Epoch: 2 itr: 1 loss: 0.00315680145286
Epoch: 2 itr: 2 loss: 0.00303068198264
Epoch: 2 itr: 3 loss: 0.00297151925042
Epoch: 2 itr: 4 loss: 0.00307324924506
Epoch: 2 itr: 5 loss: 0.00294928648509
Epoch: 2 itr: 6 loss: 0.00329540669918
Epoch: 2 itr: 7 loss: 0.00295939738862
Epoch: 2 itr: 8 loss: 0.00333702214994
Epoch: 2 itr: 9 loss: 0.00386966415681
Epoch: 2 itr: 10 loss: 0.00273666065186
Epoch: 2 itr: 11 loss: 0.00290952716023
epoch......................3
Epoch: 3 itr: 1 loss: 0.00331067317165
Epoch: 3 itr: 2 loss: 0.0028154538013
Epoch: 3 itr: 3 loss: 0.00275068939663
